<a href="https://colab.research.google.com/github/qw04/ageEstimation/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pooling Layers
2. Loss Functions
3. Optimiser
4. Metric



In [4]:
! pip install -q kaggle
from google.colab import files, output
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
import kaggle
! kaggle datasets download -d frabbisw/facial-age
! mkdir ageData1
! unzip facial-age.zip -d ageData1
! kaggle datasets download -d jangedoo/utkface-new
! mkdir ageData2
! unzip utkface-new.zip -d ageData2
! kaggle datasets download -d eshachakraborty00/all-age-face-dataset
! mkdir ageData3
! unzip all-age-face-dataset.zip -d ageData3
# output.clear()

Saving kaggle.json to kaggle (3).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
401 - Unauthorized
401 - Unauthorized
mkdir: cannot create directory ‘ageData1’: File exists
unzip:  cannot find or open facial-age.zip, facial-age.zip.zip or facial-age.zip.ZIP.
401 - Unauthorized
mkdir: cannot create directory ‘ageData2’: File exists
unzip:  cannot find or open utkface-new.zip, utkface-new.zip.zip or utkface-new.zip.ZIP.
401 - Unauthorized
mkdir: cannot create directory ‘ageData3’: File exists
unzip:  cannot find or open all-age-face-dataset.zip, all-age-face-dataset.zip.zip or all-age-face-dataset.zip.ZIP.


In [2]:
import math
import numpy as np
import os
import sys
import tensorflow as tf
from PIL import Image, features
from tqdm import tqdm
from google.colab import files, output

In [3]:
'''
categories should be inclusive
'''

# categories = [[[x,x+1] for x in range(1, 110, 1)]] #define both sides inclusive boundaries for labels
categories = [[1,17],
              [18,1000]]
              # [21,26],
              # [27,35],
              # [36,50],
              # [51,63],
              # [64,110]
              # ] #manual definition 
#folder = int("")
#print(temp)

In [11]:
data_dir1 = "/content/ageData1/face_age"
data_dir2 = "/content/ageData2/UTKFace"
data_dir3 = "/content/ageData3/All-Age-Faces Dataset/original images"
EPOCHS = 20
IMG_WIDTH = 200
IMG_HEIGHT = 200
NUM_CATEGORIES = len(categories)
TEST_SIZE = 0.1
images = []
labels = []

In [12]:
augmentedFolder = "/content/drive/MyDrive/ComputingNea"

In [13]:
'''
takes in data from the face age dataset
'''
anotherPath = "/content/drive/MyDrive/someImage.JPG"

another_useless_array = []
for folder in tqdm(os.listdir(data_dir1)):
  folder_path = os.path.join(data_dir1, folder)
  if os.path.isdir(folder_path) and folder != 'face_age':
    counter = 0
    while counter < len(categories):
      if categories[counter][0] <= int(folder) <= categories[counter][1]:
        temp = counter
        break
      counter += 1  
    for file in os.listdir(folder_path):
      try:
        if folder != 'face_age':
          image = Image.open(os.path.join(folder_path, file))
          for i in range(-10, 11, 1):
            newImage = image.rotate(i)
            newImage = image.resize((IMG_WIDTH, IMG_HEIGHT), resample=Image.BILINEAR)
            temp = f'{temp}_{i}'
            newImage.save(os.path.join(augmentedFolder, temp))
            
      except Exception as e:
        print(f'{e}: {folder}')
        another_useless_array.append(folder)
        pass

another_useless_set = set(another_useless_array)
output.clear()
print(another_useless_set)
#print(image[0])

FileNotFoundError: ignored

In [ ]:
'''
takes in data from the UTK face dataset
'''
anotherPath = "/content/drive/MyDrive/someImage.JPG"
count = 0
another_useless_array = []
for randomFile in tqdm(os.listdir(data_dir2)):
  count+=1
  if count < 21768:
    pass
  folder_path = os.path.join(data_dir2, randomFile)
  var = randomFile[:2]
  if not var.isdigit():
    var = var[0]
  counter = 0
  while counter < len(categories):
    if categories[counter][0] <= int(var) <= categories[counter][1]:
      temp = counter
      break
    counter += 1
  try:
    image = Image.open(folder_path)
    for i in range(-10, 11, 1):
    # for i in range(1):
      newImage = image.rotate(i)
      newImage = image.resize((IMG_WIDTH, IMG_HEIGHT), resample=Image.BILINEAR)
      betterImage = np.array(newImage)
      # images.append(betterImage)
      np.save(a_file, betterImage)
      labels.append(temp)
  except Exception as e:
    another_useless_array.append(randomFile)
    pass


np.savetxt(b_file, labels)
labels = []

print(set(another_useless_array))
output.clear()

100%|██████████| 23708/23708 [16:27<00:00, 24.00it/s] 


OSError: ignored

In [ ]:
anotherPath = "/content/drive/MyDrive/someImage.JPG"
useless_array = []
for randomFile in tqdm(os.listdir(data_dir3)):
  folder_path = os.path.join(data_dir3, randomFile)
  counter = 0
  while counter < len(categories):
    if categories[counter][0] <= int(randomFile[-6:-4]) <= categories[counter][1]:
      temp = counter
      break
    counter += 1
  try:
    image = Image.open(folder_path)
    for i in range(-10, 11, 1):
    # for i in range(1):
      newImage = image.rotate(i)
      newImage = image.resize((IMG_WIDTH, IMG_HEIGHT), resample=Image.BILINEAR)
      betterImage = np.array(newImage)
      # images.append(betterImage)
      # with open(output_a, 'wb') as a_file:
      np.save(a_file, betterImage)
      labels.append(temp)

  except Exception as e:
    useless_array.append(randomFile)
    pass

labels = np.array(labels)
np.savetxt(b_file, labels)
labels = []

print(set(useless_array))
output.clear()


In [ ]:
# b_file.close()
a_file.close()

In [ ]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 64, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 32, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 16, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 8, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 4, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 2, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="sigmoid"),
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [ ]:
model = get_model()
# model.summary()

In [ ]:
output_b = "/content/drive/MyDrive/labels.txt"
b_file = open(output_b, "r")
array = b_file.read()
labels = array.split('\n')
while '' in labels:
  labels.remove('')
labels = list(map(int, [i[0] for i in labels]))
labels = tf.keras.utils.to_categorical(labels)

ValueError: ignored

In [ ]:
labels = tf.keras.utils.to_categorical(labels)

ValueError: ignored

In [ ]:
trainingList = []
with open(output_a, 'rb') as f:
  for i in tqdm(range(len(labels))):
    a = np.load(f)
    trainingList.append([a, labels[i]])
  
  # output.clear()

NameError: ignored

In [ ]:
'''
finding out the number of images for different age ranges
'''
useless_array = labels.copy()
useless_set = set(useless_array)
useless_dit = []

for i in useless_set:
  useless_dit.append([i, useless_array.count(i)])

useless_dit = sorted(useless_dit ,key=lambda x: (x[0]))
x = list(map(lambda x:[int(x[0]),x[1]], useless_dit))
for i in x:
  print(f'{categories[i[0]][0]}-{categories[i[0]][1]} : {i[1]}')

1-17 : 213507
18-1000 : 769461


In [ ]:
temp = [images, labels]
temp = np.array(temp, dtype=object).T
np.random.shuffle(temp)
temp = temp.T
temp = temp.tolist()
images = temp[0]
labels = temp[1]
labels = tf.keras.utils.to_categorical(labels)
imageBuckets = [images[i:i+500] for i in range(0, len(images), 500)]
labelBuckets = [labels[i:i+500] for i in range(0, len(images), 500)]

totalBuckets = [x for x in range(len(imageBuckets))]
np.random.shuffle(totalBuckets)
variable = math.ceil(len(totalBuckets) * TEST_SIZE)
trainingBuckets = totalBuckets[:variable]
testingBuckets = totalBuckets[variable+1:]

In [ ]:
np.array(imageBuckets[0]).shape

(500, 200, 200, 3)

In [ ]:
for e in range(EPOCHS):
  np.random.shuffle(trainingBuckets)
  for i in trainingBuckets:
    model.fit(np.array(imageBuckets[i]), np.array(labelBuckets[i]))
  output.clear()

 8/16 [==============>...............] - ETA: 7s - loss: 0.5656 - accuracy: 0.6367

KeyboardInterrupt: ignored

In [ ]:
np.random.shuffle(testingBuckets)
for i in testingBuckets:
    model.evaluate(np.array(imageBuckets[i]),  np.array(labelBuckets[i]), verbose=2)

In [ ]:
for i in trainingBuckets:
  print(set(labelBuckets[i]))

TypeError: ignored

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/model')